In [ ]:
# Import Libraries
import numpy as np
import pandas as pd

In [ ]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
  
# fetch dataset 
soybean_large = fetch_ucirepo(id=90) 
  
# data (as pandas dataframes) 
X = soybean_large.data.features 
y = soybean_large.data.targets 
  
# Display columns
print(X.head())
print(y.head())

   date  plant-stand  precip  temp  hail  crop-hist  area-damaged  severity  \
0   6.0          0.0     2.0   1.0   0.0        1.0           1.0       1.0   
1   4.0          0.0     2.0   1.0   0.0        2.0           0.0       2.0   
2   3.0          0.0     2.0   1.0   0.0        1.0           0.0       2.0   
3   3.0          0.0     2.0   1.0   0.0        1.0           0.0       2.0   
4   6.0          0.0     2.0   1.0   0.0        2.0           0.0       1.0   

   seed-tmt  germination  ...  int-discolor  sclerotia  fruit-pods  \
0       0.0          0.0  ...           0.0        0.0         0.0   
1       1.0          1.0  ...           0.0        0.0         0.0   
2       1.0          2.0  ...           0.0        0.0         0.0   
3       0.0          1.0  ...           0.0        0.0         0.0   
4       0.0          2.0  ...           0.0        0.0         0.0   

   fruit-spots  seed  mold-growth  seed-discolor  seed-size  shriveling  roots  
0          4.0   0.0   

In [28]:
from factor_analyzer.factor_analyzer import calculate_kmo
from sklearn.linear_model import LogisticRegression


# Process the data
X_processed = X.copy()

# Fill missing values with mean
X_processed = X_processed.fillna(X_processed.mean())

# Encode target variable (disease classes)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y.values.ravel())

# Split data and scale features
X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y_encoded, test_size=0.3, random_state=42
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build model without PCA 
model = LogisticRegression()
model.fit(X_train_scaled, y_train)
accuracy = accuracy_score(y_test, model.predict(X_test_scaled))
print(f"Accuracy (Logistic Regression): {accuracy:.4f}")

# KMO Test
kmo_all,kmo_model=calculate_kmo(X_train_scaled)
print(f"KMO: {kmo_model:.4f}")

# Perform PCA
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Perform Regression with PC as predictors
pca_model = LogisticRegression()
pca_model.fit(X_train_pca, y_train)
pca_accuracy = accuracy_score(y_test, pca_model.predict(X_test_pca))
print(f"PCA accuracy (Logistic Regression): {pca_accuracy:.4f}")


Accuracy (Logistic Regression): 0.8817
KMO: 0.6166
PCA accuracy (Logistic Regression): 0.8925


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/factor_analyzer/utils.py:244: UserWarning: The inverse of the variance-covariance matrix was calculated using the Moore-Penrose generalized matrix inversion, due to its determinant being at or very close to zero.
  warnings.warn(
